In [1]:
'''
Train pipeline and save best result
'''
##### Add arguments #####
import argparse
#########################

##### custom lib and metrics #####
from password_complexity.metrics.metrics import rmsle_score, RMSLE_score, RMSLE
from password_complexity.features.generate import generate_features
from password_complexity.utils.config import load_config
from password_complexity.metrics.metrics import *
from password_complexity.utils.hyperparameters import params_distribution
from password_complexity.utils.pipeline_structure import create_pipeline_structure
#################################

##### Preprocessing #####
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
#########################

##### Path lib #####
import os
from pathlib import Path
import joblib
####################

##### dataset instruments #####
import pandas as pd
from sklearn.model_selection import train_test_split
###############################

##### Model structure #####
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
###########################

##### Models #####
from sklearn.linear_model import LinearRegression
##################

##### hyperparams optimize #####
import optuna
from optuna.integration import OptunaSearchCV
from optuna.distributions import *
################################

In [2]:

config_ = load_config('../../CONFIG_PATH.yaml')
print('загружаем конфиг', config_)
df=pd.read_csv('../../'+config_.make_dataset.train_dataset)

df=generate_features(df)
print('читаем датасет и генерим фичи')



загружаем конфиг {'base': {'random_state': 42, 'log_level': 'DEBUG'}, 'make_dataset': {'train_dataset': 'data/train.csv', 'test_dataset': 'data/Xtest.csv', 'train_features_interim': 'data/interim/train_features.feather', 'train_target_processed': 'data/processed/train_target.feather', 'test_features_interim': 'data/interim/test_features.feather'}, 'build_features': {'train_features_processed': 'data/processed/train_features.feather', 'test_features_processed': 'data/processed/test_features.feather'}, 'train_model': {'model_path': 'models/model_config.joblib', 'train_size': 0.67}}
читаем датасет и генерим фичи


In [3]:
y = df['Times']
X = df.drop('Times', axis=1)
print('разбиваем на таргет и лейбл')

разбиваем на таргет и лейбл


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,y, 
                                                    train_size=config_.train_model.train_size, 
                                                    random_state=config_.base.random_state
                                                    )
print('разбиваем на обучение и тест', X_train.shape, X_test.shape)

разбиваем на обучение и тест (2781502, 14) (1369994, 14)


In [5]:
# model = joblib.load('../../'+config_.train_model.model_path)
# импортируем старую модель
print('импортирем старую модель')

## Создаем списки коннилчественных
## и категориальных столбцов
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index

pipe = create_pipeline_structure(num_columns, cat_columns)
print('Создаем пайплайн для обучения', pipe)

импортирем старую модель
Создаем пайплайн для обучения Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['password_length', 'unique_chars_count', 'unique_chars_prop',
       'is_alnum', 'is_alpha', 'is_lower', 'is_upper', 'is_numeric',
       'letters_count', 'letters_prob', 'difits_count', 'digits_prob',
       'is_lower_and_upper', 'palindrom'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   Si

In [8]:
optuna_search = OptunaSearchCV(
                            pipe,
                            params_distribution,
                            scoring=RMSLE_score,
                            random_state=config_.base.random_state,
                            n_trials=10,
                            verbose=1,
                            cv=5
                            )

/tmp/ipykernel_114782/3981927596.py:1: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = OptunaSearchCV(


In [10]:
optuna_search.fit(X_train.iloc[1660:1765], y_train.iloc[1660:1765])


/home/nikita/.pyenv/versions/3.9.6/envs/password_comp/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classifier=False as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
[I 2021-09-16 19:59:39,096] A new study created in memory with name: no-name-547cbc55-00b4-4552-821b-ad074daf39cd
[I 2021-09-16 19:59:39,097] Searching the best hyperparameters using 105 samples...
[I 2021-09-16 19:59:39,958] Trial 0 finished with value: -0.851147435192255 and parameters: {'transform__cat__imputer__strategy': 'median'}. Best is trial 0 with value: -0.851147435192255.
[I 2021-09-16 19:59:40,923] Trial 1 finished with value: -0.821809387357084 and parameters: {'transform__cat__imputer__strategy': 'median'}. Best is trial 1 with value: -0.821809387357084.
[I 2021-09-16 19:59:42,024] Trial 2 finished with value: -0.8401375381139833 and parameters:

OptunaSearchCV(estimator=Pipeline(steps=[('transform',
                                          ColumnTransformer(transformers=[('num',
                                                                           Pipeline(steps=[('imputer',
                                                                                            SimpleImputer()),
                                                                                           ('scaler',
                                                                                            StandardScaler())]),
                                                                           Index(['password_length', 'unique_chars_count', 'unique_chars_prop',
       'is_alnum', 'is_alpha', 'is_lower', 'is_upper', 'is_numeric',
       'letters_count', 'letters_prob', 'difits_count', 'digits_prob',...
                                                                           Pipeline(steps=[('imputer',
                                             

In [16]:
X_train.iloc[1655:1665]

,password_length,unique_chars_count,unique_chars_prop,is_alnum,is_alpha,is_lower,is_upper,is_numeric,letters_count,letters_prob,difits_count,digits_prob,is_lower_and_upper,palindrom
2023443,8,7,0.875000,1,1,1,0,0,8,1.000000,0,0.000000,1,False
3784793,8,8,1.000000,1,0,0,1,0,6,0.750000,2,0.250000,1,False
3407005,8,7,0.875000,1,1,1,0,0,8,1.000000,0,0.000000,1,False
2194342,6,6,1.000000,1,0,1,0,0,4,0.666667,2,0.333333,1,False
3156615,12,8,0.666667,1,1,0,0,0,12,1.000000,0,0.000000,0,False
1264267,4,4,1.000000,1,1,1,0,0,4,1.000000,0,0.000000,1,False
3596620,5,4,0.800000,1,1,0,1,0,5,1.000000,0,0.000000,1,False
1124136,6,3,0.500000,1,1,1,0,0,6,1.000000,0,0.000000,1,False
2776250,6,4,0.666667,1,0,1,0,0,1,0.166667,5,0.833333,1,False
195413,10,8,0.800000,1,0,1,0,0,6,0.600000,4,0.400000,1,False


In [11]:
y_train.iloc[1655:1665]

2023443      1
3784793      1
3407005      2
2194342      1
3156615      1
1264267    177
3596620      1
1124136      1
2776250      1
195413       1
Name: Times, dtype: int64

In [12]:
optuna_search.best_estimator_.predict(X_train.iloc[1655:1665])

array([  1.        ,   1.04      ,   1.        ,   1.49      ,
         1.39      , 120.7       ,   4.66      ,   1.38738095,
         1.18      ,   1.        ])